### Evaluating the m5 limits ###

Create values to be used in `rubin_sim.utils.sysEngVals`

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import syseng_throughputs as st

In [2]:
pd.set_option('display.precision', 3)

In [3]:
# Read bandpass information.
defaultDirs = st.setDefaultDirs()
addLosses = True
# Use the X=1.0 atmosphere!
atmosphere = st.readAtmosphere(defaultDirs['atmosphere'], atmosFile='atmos_10_aerosol.dat')
hardware, system = st.buildHardwareAndSystem(defaultDirs, addLosses, atmosphereOverride=atmosphere)

/Users/lynnej/lsst_repos/rubin_sim/rubin_sim/phot_utils/bandpass.py:100: UserWarning: Wavelength sampling of 1.0 nm is > 0.2 nm, this may not work well with a Sed object. Consider resampling with resample_bandpass method.
  warnings.warn(
/Users/lynnej/lsst_repos/rubin_sim/rubin_sim/phot_utils/bandpass.py:100: UserWarning: Wavelength sampling of 800.0 nm is > 0.2 nm, this may not work well with a Sed object. Consider resampling with resample_bandpass method.
  warnings.warn(


In [4]:
# --note these are 1x30s and the overview paper uses 2x15s!!! 
# (this is used as the 'base exposure time' in sysEngVals and the m5 calculation from Cm/dCm_infinity)
# Note also this is using fiducial (old) values of the seeing to set the m5 values as well. 
# thus the resulting values are different than those reported in pstn-054, where the Cm/dCm reported values are 
# for the median delivered seeing in the opsim databases.

m5_std = st.makeM5(hardware, system, readnoise=8.8, othernoise=0, darkcurrent=0.2)

m5_std

,nexp,exptime,FWHMeff,FWHMgeom,skyMag,skyCounts,Zp_t,Tb,Sb,kAtm,gamma,Cm,dCm_infinity,dCm_double,m5,sourceCounts,m5_fid,m5_min
u,1.0,30.0,0.92,0.808,22.948,73.638,26.911,0.033,0.052,0.507,0.038,23.302,0.397,0.237,23.979,446.606,23.9,23.4
g,2.0,15.0,0.87,0.767,22.265,420.441,28.411,0.130,0.159,0.216,0.039,24.447,0.173,0.091,24.843,801.999,25.0,24.6
r,2.0,15.0,0.83,0.734,21.200,834.660,28.179,0.105,0.118,0.127,0.039,24.459,0.094,0.047,24.374,998.126,24.7,24.3
i,2.0,15.0,0.80,0.710,20.470,1250.952,27.919,0.083,0.091,0.096,0.039,24.376,0.065,0.031,23.966,1143.800,24.0,23.6
z,2.0,15.0,0.78,0.693,19.599,1916.456,27.538,0.058,0.062,0.069,0.039,24.223,0.043,0.020,23.405,1350.441,23.3,22.9
y,2.0,15.0,0.76,0.677,18.598,2368.700,26.667,0.026,0.031,0.169,0.039,23.746,0.035,0.016,22.456,1451.047,22.1,21.7


In [5]:
# Write out values of dictionaries that can be pasted into sims_utils
output_cols = ['Zp_t', 'Tb', 'gamma', 'kAtm', 'Cm', 'dCm_infinity', 'dCm_double', 'skyMag']
for column in output_cols:
    result = '{'
    for filtername, val in zip(m5_std.index, m5_std[column]):
        result += '"%s": %f, ' % (filtername, val)
    result += '}'
    result = result.replace(', }', '}')
    result = 'self.'+column+ ' = ' + result
    print(result)

self.Zp_t = {"u": 26.911304, "g": 28.410753, "r": 28.178815, "i": 27.919362, "z": 27.537939, "y": 26.666904}
self.Tb = {"u": 0.032748, "g": 0.130306, "r": 0.105242, "i": 0.082872, "z": 0.058323, "y": 0.026147}
self.gamma = {"u": 0.037761, "g": 0.038753, "r": 0.038998, "i": 0.039126, "z": 0.039260, "y": 0.039311}
self.kAtm = {"u": 0.506813, "g": 0.215549, "r": 0.126720, "i": 0.095903, "z": 0.068894, "y": 0.169114}
self.Cm = {"u": 23.301865, "g": 24.446678, "r": 24.458747, "i": 24.376211, "z": 24.222775, "y": 23.745944}
self.dCm_infinity = {"u": 0.396871, "g": 0.172866, "r": 0.094418, "i": 0.064930, "z": 0.043343, "y": 0.035358}
self.dCm_double = {"u": 0.237427, "g": 0.091121, "r": 0.046826, "i": 0.031034, "z": 0.019885, "y": 0.015820}
self.skyMag = {"u": 22.948308, "g": 22.264991, "r": 21.199715, "i": 20.470117, "z": 19.598539, "y": 18.597696}
